In [2]:
import numpy as np
from scipy.stats import norm
from copy import copy

## Problem 2

In [3]:
class GBM:

    def __init__(self,sigma,r,drift,S_t,t):
        self.sigma = sigma
        self.r = r
        self.drift = drift   
        self.S_t = S_t
        self.t = t

In [4]:
class CallOption:

    def __init__(self, K, T):
        self.K = K
        self.T = T

In [5]:
class CallBinary:

    def __init__(self, K, T):
        self.K = K
        self.T = T

In [6]:
class EYcontract:

    def __init__(self,threshold0,K_case1,K_case2lower,K_case2upper,T0,T1):
        self.K_case1 = K_case1
        self.K_case2lower = K_case2lower
        self.K_case2upper = K_case2upper
        self.threshold0 = threshold0
        self.T0=T0
        self.T1=T1

In [7]:
class AnalyticEngine:

    def __init__(self):
        pass

    def BSpriceCall(self, dynamics, contract):
        timeRemaining = contract.T - dynamics.t
        F = dynamics.S_t*np.exp(dynamics.drift*timeRemaining)
        std = dynamics.sigma*np.sqrt(timeRemaining)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        return np.exp(-dynamics.r*timeRemaining)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))

    def BSpriceCallBinary(self, dynamics, contract):

        timeRemaining = contract.T - dynamics.t
        F = dynamics.S_t*np.exp(dynamics.drift*timeRemaining)
        std = dynamics.sigma*np.sqrt(timeRemaining)
        d2 = np.log(F/contract.K)/std+std/2
        return np.exp(-dynamics.r*timeRemaining)*norm.cdf(d2)

In [8]:
class MCengine:

    def __init__(self, M, seed):
        self.M = M                                   
        self.rng = np.random.default_rng(seed=seed) 

    def price_EYcontract(self,contract,dynamics):
        M,sigma,r,drift,S_t,t = self.M,dynamics.sigma,dynamics.r,dynamics.drift,dynamics.S_t,dynamics.t
        dt = contract.T0 - t
        Z = self.rng.normal(size=M)
        S_T = S_t*np.exp((drift-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z)
        payoff = np.zeros(M)


        for i in range(M):

            embeddedCallcase1 = CallOption(K=contract.K_case1, T=contract.T1)
            embeddedCallcase2 = CallOption(K=contract.K_case2lower, T=contract.T1)
            embeddedBinarycase2 = CallBinary(K=contract.K_case2upper, T=contract.T1)
            dynamicsConditional = copy(dynamics)
            dynamicsConditional.t = contract.T0

            if S_T[i] > contract.threshold0:
                dynamicsConditional.S_t = S_T[i]
                payoff[i] = AnalyticEngine().BSpriceCall(dynamicsConditional,embeddedCallcase1) 
            else:
                dynamicsConditional.S_t = S_T[i]
                payoff[i] = AnalyticEngine().BSpriceCall(dynamicsConditional,embeddedCallcase2) + AnalyticEngine().BSpriceCallBinary(dynamicsConditional,embeddedBinarycase2)
            payoff[i] = np.exp(-r*dt)*payoff[i]
        price = np.mean(payoff)
        standard_error = np.std(payoff)/np.sqrt(M)


        return(price, standard_error)


In [9]:
hw7p2contract=EYcontract(threshold0=12,K_case1=11,K_case2lower=10,K_case2upper=14,T0=0.5,T1=1.0)

In [10]:
hw7p2dynamics=GBM(sigma=0.7,r=0.02,drift=0.02,S_t=10,t=0)

In [11]:
hw7p2MC=MCengine(M=100000,seed=0)
(price, std_err) =  hw7p2MC.price_EYcontract(hw7p2contract,hw7p2dynamics)
print(price,std_err)

2.7432825465309123 0.01130510426676067
